# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [2]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [3]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 1921


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [4]:
# YOUR CHANGES HERE
import pandas as pd

def random_policy(state, actions):
    return random.choice(actions)

Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [5]:
# YOUR CHANGES HERE

rows = []

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
    rows.append({
        "curr_state": curr_state,
        "curr_action": curr_action,
        "next_reward": next_reward,
        "next_state": next_state
    })




pd.DataFrame(rows, columns=["curr_state","curr_action","next_reward","next_state"])\
  .to_csv("log-random.tsv", sep="\t", index=False)

CURR STATE 2045 ACTION -1 NEXT REWARD 0 NEXT STATE 1908
CURR STATE 1908 ACTION -1 NEXT REWARD 0 NEXT STATE 1779
CURR STATE 1779 ACTION 1 NEXT REWARD 0 NEXT STATE 1660
CURR STATE 1660 ACTION 0 NEXT REWARD 0 NEXT STATE 1532
CURR STATE 1532 ACTION -1 NEXT REWARD 0 NEXT STATE 1403
CURR STATE 1403 ACTION -1 NEXT REWARD 0 NEXT STATE 1274
CURR STATE 1274 ACTION 1 NEXT REWARD 0 NEXT STATE 1275
CURR STATE 1275 ACTION 1 NEXT REWARD 0 NEXT STATE 1148
CURR STATE 1148 ACTION -1 NEXT REWARD 0 NEXT STATE 1019
CURR STATE 1019 ACTION -1 NEXT REWARD 0 NEXT STATE 890
CURR STATE 890 ACTION 0 NEXT REWARD 0 NEXT STATE 890
CURR STATE 890 ACTION -1 NEXT REWARD 0 NEXT STATE 889
CURR STATE 889 ACTION 0 NEXT REWARD 0 NEXT STATE 1017
CURR STATE 1017 ACTION 1 NEXT REWARD 0 NEXT STATE 1146
CURR STATE 1146 ACTION 0 NEXT REWARD 0 NEXT STATE 1146
CURR STATE 1146 ACTION 0 NEXT REWARD 0 NEXT STATE 1146
CURR STATE 1146 ACTION -1 NEXT REWARD 0 NEXT STATE 1145
CURR STATE 1145 ACTION 1 NEXT REWARD 0 NEXT STATE 1274
CURR STA

Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [6]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
        pass

Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [7]:
# YOUR CHANGES HERE

alpha = 1.0
gamma = 0.9

Q = {}

def q_value(state, action):
    return Q.get((state, action), 0.0)

def max_q(state):
    return max((q_value(state, a) for a in simulator.get_actions(state)), default=0.0)

rows_2 = []

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        old_value = q_value(curr_state, curr_action)
        target = next_reward + gamma * max_q(next_state)
        new_value = (1 - alpha) * old_value + alpha * target
        Q[(curr_state, curr_action)] = new_value

        rows_2.append({
            "curr_state": curr_state,
            "curr_action": curr_action,
            "next_reward": next_reward,
            "next_state": next_state,
            "old_value": old_value,
            "new_value": new_value
        })

df = pd.DataFrame(rows_2, columns=["curr_state", "curr_action", "next_reward", "next_state", "old_value", "new_value"])
df.to_csv("q-random.tsv", sep="\t", index=False)

Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [8]:
# YOUR CHANGES HERE

# hard-code epsilon=0.25. this is high but the environment is deterministic.
epsilon = 0.25
A = [-1, 0, 1]

Q = {}
q     = lambda s,a: Q.get((s,a), 0.0)
max_q = lambda s: max((q(s,a) for a in A), default=0.0)

def eps_greedy(s):
    if random.random() < epsilon:
        return random.choice(A)
    vals = [(a, q(s,a)) for a in A]
    m = max(v for _,v in vals)
    best = [a for a,v in vals if v == m]
    return random.choice(best)



Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [9]:
# YOUR CHANGES HERE

rows_3 = []
seen_pairs = set()
episodes  = 32
max_steps = 1024


for _ in range(episodes):

     for (s, a, r, s2) in simulator.rollout_policy(lambda st, acts: eps_greedy(st), max_steps=max_steps):
        old = q(s,a)
        new = r + gamma * max_q(s2)
        Q[(s,a)] = new 
        rows_3.append({
            "curr_state": s,
            "curr_action": a,
            "next_reward": r,
            "next_state": s2,
            "old_value": old,
            "new_value": new
        })
df = pd.DataFrame(rows_3, columns=[
    "curr_state", "curr_action", "next_reward", "next_state", "old_value", "new_value"
])
df.to_csv("q-greedy.tsv", sep="\t", index=False)


Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [10]:
# YOUR CHANGES HERE

df = pd.read_csv("q-greedy.tsv", sep="\t")[["curr_state", "curr_action", "new_value"]]
df.columns = ["state", "action", "q"]

wide = df.pivot_table(index="state",
                      columns="action",
                      values="q",
                      aggfunc="last").fillna(0.0)
wide = wide.reindex(columns=[0, -1, 1])

best = wide.idxmax(axis=1).rename("action").reset_index()

best["state"] = best["state"].astype(int)
best["action"] = best["action"].astype(int)
best = best.sort_values("state", kind="mergesort").reset_index(drop=True)

best.to_csv("policy-greedy.tsv", sep="\t", index=False)

Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [17]:
# YOUR CHANGES HERE

A = [-1, 0, 1]

sim = RoverSimulator(16)

cx, cy = sim.resolution // 2, sim.resolution // 2

def q_learning_optimal(state, a):
    a = int(a); state = int(state)
    _, s1 = sim.get_next_reward_state(state, a)
    x1, y1, d1 = sim.decode_state(int(s1))
    dx1, dy1 = RoverSimulator.DIRECTIONS[int(d1)]
    nx = min(max(0, x1 + dx1), sim.resolution - 1)
    ny = min(max(0, y1 + dy1), sim.resolution - 1)
    return max(abs(cx - nx), abs(cy - ny))

g = pd.read_csv("q-greedy.tsv", sep="\t")
qa = (g[["curr_state","curr_action","new_value"]]
        .rename(columns={"curr_state":"state","curr_action":"action","new_value":"q"})
        .groupby(["state","action"], as_index=False)
        .tail(1))

qa["state"]  = qa["state"].astype(int)
qa["action"] = qa["action"].astype(int)

Q = {(int(r.state), int(r.action)): float(r.q) for r in qa.itertuples(index=False)}
q = lambda s,a: Q.get((s,a), 0.0)

rows_5 = []
num_states = sim.resolution * sim.resolution * 8

for s in range(num_states):
    v0, vL, vR = q(s,0), q(s,-1), q(s,1)
    m, M = min(v0, vL, vR), max(v0, vL, vR)

    if M > m:
        if v0 == M: a_star = 0
        elif vL == M: a_star = -1
        else: a_star = 1
    else:
        pref_rank = {0:0, -1:1, 1:2}
        a_star = min(A, key=lambda a: (q_learning_optimal(s, a), pref_rank[a]))

    rows_5.append({"state": int(s), "action": int(a_star)})

out = pd.DataFrame(rows_5, columns=["state","action"]).fillna(0)
out["state"]  = out["state"].astype(int)
out["action"] = out["action"].astype(int)

out.to_csv("policy-optimal.tsv", sep="\t", index=False)

Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

None